In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS']='1'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from tf_keras.optimizers import Adam
from tf_keras.losses import SparseCategoricalCrossentropy
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

2025-12-23 19:06:03.887210: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-23 19:06:03.922669: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-23 19:06:04.647726: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
tokenzier=AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
df=pd.read_csv('/home/chinghia/AI_Engineer/IMDB_Dataset/IMDB Dataset.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [7]:
X=df.drop('sentiment',axis=1)
y=df['sentiment']
X.head()

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."


In [8]:
def clean_code(text):
    text=re.sub(r'<.*?>',' ',text)
    text=re.sub(r'http\S+|www\S+|https\S+', ' ', text, flags=re.MULTILINE)
    text=re.sub(r'\s+',' ',text).strip()
    return text

In [9]:
X['review']=X['review'].apply(clean_code)
X.head(5)

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."


In [10]:
X=X.iloc[:,0]  #[x ,y:z ,e]
y=y.astype('category').cat.codes

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
def encode_data(text):
    encodings=tokenzier(
        text.tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='tf'
    )
    return encodings

In [13]:
X_train=encode_data(X_train)
X_test=encode_data(X_test)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
I0000 00:00:1766491579.195379   11510 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2128 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [14]:
modeltrans=TFAutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    use_safetensors=False)

tf_model.h5:   0%|          | 0.00/363M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-

In [15]:
modeltrans.compile(
    optimizer=Adam(learning_rate=2e-5),#0.00002
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [16]:
modeltrans.fit(
    dict(X_train),y_train,
    epochs=1,
    batch_size=2,
    validation_data=(dict(X_test),y_test)
)

2025-12-23 19:07:07.774807: I external/local_xla/xla/service/service.cc:163] XLA service 0x73cac2a640c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-23 19:07:07.774821: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-12-23 19:07:07.783094: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-23 19:07:07.808594: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701
I0000 00:00:1766491627.851639   11817 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20000/20000 [==============================] - 788s 39ms/step - loss: nan - accuracy: 0.7848 - val_loss: nan - val_accuracy: 0.4961


In [19]:
modeltrans.evaluate(X_test,y_test)

313/313 [==============================] - 37s 117ms/step - loss: nan - accuracy: 0.4961


[nan, 0.4961000084877014]